In [8]:
import os
os.environ["RPY2_CFFI_MODE"] = "ABI"
from rpy2 import robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
import pandas as pd

base = importr("base")
print(ro.r("R.version.string"))

[1] "R version 4.5.2 (2025-10-31 ucrt)"



In [9]:
# 4) Load renv and project library
renv = importr("renv")
ro.r("renv::load()")   # more reliable than renv.load() in some setups

# 5) Load mgcv and fit
mgcv = importr("mgcv")
fit = ro.r("mgcv::gam(Sepal.Length ~ s(Sepal.Width), data=iris)")
print(ro.r("summary")(fit))

- Project 'C:/Users/yimingz/repo/quantbullet' loaded. [renv 1.1.5]

Family: gaussian 
Link function: identity 

Formula:
Sepal.Length ~ s(Sepal.Width)

Parametric coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  5.84333    0.06524   89.56   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Approximate significance of smooth terms:
                 edf Ref.df     F p-value  
s(Sepal.Width) 3.592  4.505 2.635   0.032 *
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

R-sq.(adj) =  0.0688   Deviance explained = 9.13%
GCV = 0.65866  Scale est. = 0.6385    n = 150



In [10]:
import pandas as pd
from rpy2 import robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

# ensure renv is loaded already
importr("renv")
ro.r("renv::load()")
importr("mgcv")

df = pd.DataFrame({"x": range(1, 201)})
df["y"] = (df["x"] ** 0.5) + 0.1 * (df["x"] % 7)

# Convert pandas -> R safely
with localconverter(ro.default_converter + pandas2ri.converter):
    r_df = ro.conversion.py2rpy(df)

ro.globalenv["df"] = r_df

fit = ro.r("mgcv::gam(y ~ s(x, k=20), data=df)")
print(ro.r("summary")(fit))

# Convert prediction back to numpy (better than list for real work)
pred = ro.r("predict")(fit)
pred_np = ro.vectors.FloatVector(pred)  # or use numpy converter below
print("First 5 preds:", list(pred)[:5])


- Project 'C:/Users/yimingz/repo/quantbullet' loaded. [renv 1.1.5]

Family: gaussian 
Link function: identity 

Formula:
y ~ s(x, k = 20)

Parametric coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  9.76142    0.01498   651.7   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Approximate significance of smooth terms:
      edf Ref.df    F p-value    
s(x) 11.7  14.12 3461  <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

R-sq.(adj) =  0.996   Deviance explained = 99.6%
GCV = 0.047913  Scale est. = 0.04487   n = 200

First 5 preds: [1.7152456976668056, 1.9041760570566257, 2.092829609427106, 2.2807803252446632, 2.4674656653496108]


In [21]:
fit.names

'coeffici...,'residuals','fitted.v...,...,'min.edf','optimizer','call'


In [24]:
fit.names[ 0 ]

'coefficients'

In [31]:
fit.rx2['coefficients']

9.761421,1.402605,4.568891,...,-0.646214,8.441697,6.458063


In [37]:
for i in range(len(fit.names)):
    print(fit.names[i])

coefficients
residuals
fitted.values
family
linear.predictors
deviance
null.deviance
iter
weights
prior.weights
df.null
y
converged
sig2
edf
edf1
hat
R
boundary
sp
nsdf
Ve
Vp
rV
mgcv.conv
gcv.ubre
aic
rank
gcv.ubre.dev
scale.estimated
method
smooth
full.sp
formula
var.summary
cmX
model
control
terms
pred.formula
pterms
assign
offset
df.residual
min.edf
optimizer
call
